# Master Discogs Database

In [13]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import discConv, isKnownAlbum, directoryName
from masterdb import getSlimArtistDB, getArtistNameToIDMap
from masterdb import getMusicData, getRowByIndex

### General
from discogsBase import discogs

### Discogs
from artistsDC import artistsDC
from artistDC import artistDC
from discogsUtils import discogsUtils
from albums import albums
from album import album, albumURLInfo

### AllMusic
from artistsAM import artistsAM
from artistAM import artistAM
from discogsUtils import allmusicUtils

### MusicBrainz
from artistsMB import artistsMB
from artistMB import artistMB
from discogsUtils import musicbrainzUtils

## AceBootlegs
from artistAB import artistAB
from artistsAB import artistsAB
from discogsUtils import acebootlegsUtils

## RateYourMusic
from artistRM import artistRM
from artistsRM import artistsRM
from discogsUtils import rateyourmusicUtils

## LastFM
from artistLM import artistLM
from artistsLM import artistsLM
from discogsUtils import lastfmUtils

## DatPiff
from artistDP import artistDP
from artistsDP import artistsDP
from discogsUtils import datpiffUtils

## RockCorner
from artistRC import artistRC
from artistsRC import artistsRC
from discogsUtils import rockcornerUtils

## CDandLP
from artistCL import artistCL
from artistsCL import artistsCL
from discogsUtils import cdandlpUtils


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-03 20:06:12.589701


In [14]:
%load_ext autoreload
%autoreload

discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artistsDC(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

discAB   = discogs('acebootlegs')
artsAB   = artistsAB(discAB)
dutilsAB = acebootlegsUtils()

discRM   = discogs('rateyourmusic')
artsRM   = artistsRM(discRM)
dutilsRM = rateyourmusicUtils()

discLM   = discogs('lastfm')
artsLM   = artistsLM(discLM)
dutilsLM = lastfmUtils()

discDP   = discogs('datpiff')
artsDP   = artistsDP(discDP)
dutilsDP = datpiffUtils()

discRC   = discogs('rockcorner')
artsRC   = artistsRC(discRC)
dutilsRC = rockcornerUtils()

discCL   = discogs('cdandlp')
artsCL   = artistsRC(discCL)
dutilsCL = cdandlpUtils()

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Apr 03, 2020 20:06:30 for Last Run


# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

#### AllMusic

In [ ]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discAM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistAMDB = getSlimArtistDB(discAM, force=True)
discSlimArtistAMDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistAMAlbumsDB = getArtistAlbumsDB(discAM, force=True)
discArtistAMAlbumsDB.head()

#### MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discMB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistMBDB = getSlimArtistDB(discMB, force=True)
discSlimArtistMBDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discMB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistMBAlbumsDB = getArtistAlbumsDB(discMB, force=True)
discArtistMBAlbumsDB.head()

In [ ]:
cntr = Counter()
for k,v in discArtistMBAlbumsDB["Albums"].to_dict().items():
    for k2,v2 in v.items():
        cntr[k2] += len(v2)
cntr.most_common()

In [ ]:
if False:
    import json
    keys = [x[0] for x in cntr.most_common()]
    keys = [x for x in keys if x not in primary]
    keys = [x for x in keys if x not in secondary]
    keys = [x for x in keys if x not in tertiary]
    keys = [x for x in keys if x not in fourth]
    #tertiary = [x for x in keys if x.startswith("Single") or x.startswith("EP")]
    #print(json.dumps(fourth))
    #json.dumps(tertiary)
    #secondary = [x for x in keys if x.find("Compilation") != -1]
    #json.dumps(keys)
    #keys

#### Discogs

In [ ]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discDC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistDCDB = getSlimArtistDB(discDC, force=True)
discSlimArtistDCDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discDC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistDCAlbumsDB = getArtistAlbumsDB(discDC, force=True)
discArtistDCAlbumsDB.head()

#### AceBootlegs

In [ ]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discAB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistABDB = getSlimArtistDB(discAB, force=True)
discSlimArtistABDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistABAlbumsDB = getArtistAlbumsDB(discAB, force=True)
discArtistABAlbumsDB.head()

#### RateYourMusic

In [ ]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discRM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistRMDB = getSlimArtistDB(discRM, force=True)
discSlimArtistRMDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discRM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistRMAlbumsDB = getArtistAlbumsDB(discRM, force=True)
discArtistRMAlbumsDB.head()

#### LastLM

In [15]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discLM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistLMDB = getSlimArtistDB(discLM, force=True)
discSlimArtistLMDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discLM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistLMAlbumsDB = getArtistAlbumsDB(discLM, force=True)
discArtistLMAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Apr 03, 2020 20:06:40 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/0-Metadata.p  	0 453
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/1-Metadata.p  	1 917
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/10-Metadata.p  	2 1409
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/11-Metadata.p  	3 1881
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/12-Metadata.p  	4 2373
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/13-Metadata.p  	5 2891
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/14-Metadata.p  	6 3368
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/15-Metadata.p  	7 3848
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/16-Metadata.p  	8 4339
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/17-Metadata.p  	9 4782
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/18-Metadata.p  	10 5244
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/19-Metad

  --> This file is 1.6MB.
Saved data to /Volumes/Biggy/Discog/db-lastfm/ArtistIDToVariations.p
  --> This file is 1.6MB.
Current Time is Fri Apr 03, 2020 20:06:45 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 5 seconds.
Current Time is Fri Apr 03, 2020 20:06:45 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 48049 Artists
	Shape --> (48049, 1)
  Finding Real Artist Name
	Shape --> (48049, 3)
  Removing None Artist
	Shape --> (48049, 3)
  Finding Disc Artist Name
	Shape --> (48049, 4)
DataFrame Shape is (48049, 4)
Current Time is Fri Apr 03, 2020 20:06:46 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 1 seconds.
Saving Master Artist DB File
  --> This file is 6.2MB.
Current Time is F

/Volumes/Biggy/Discog/artists-lastfm-db/metadata/78-MediaMetadata.p 	36982     57264     57264     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/79-MediaMetadata.p 	37468     57942     57942     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/8-MediaMetadata.p 	37940     58748     58748     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/80-MediaMetadata.p 	38434     59596     59596     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/81-MediaMetadata.p 	38894     60035     60035     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/82-MediaMetadata.p 	39417     60615     60615     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/83-MediaMetadata.p 	39879     61233     61233     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/84-MediaMetadata.p 	40391     62048     62048     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/85-MediaMetadata.p 	40905     62637     62637     
/Volumes/Biggy/Discog/artists-lastfm-db/metadata/86-MediaMetadata.p 	41413     63433     63433     
/

,Albums
10074300700,{'Albums': {}}
10197954100,{'Albums': {}}
10588500900,{'Albums': {}}
10998118200,{'Albums': {}}
11060537400,{'Albums': {}}


#### DatPiff

In [ ]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discDP)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistDPDB = getSlimArtistDB(discDP, force=True)
discSlimArtistDPDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discDP)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistDPAlbumsDB = getArtistAlbumsDB(discDP, force=True)
discArtistDPAlbumsDB.head()

#### RockCorner

In [16]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discRC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistRCDB = getSlimArtistDB(discRC, force=True)
discSlimArtistRCDB.head()


####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discRC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistRCAlbumsDB = getArtistAlbumsDB(discRC, force=True)
discArtistRCAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Apr 03, 2020 20:07:03 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/0-Metadata.p  	0 19
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/1-Metadata.p  	1 30
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/10-Metadata.p  	2 45
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/11-Metadata.p  	3 59
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/12-Metadata.p  	4 72
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/13-Metadata.p  	5 83
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/14-Metadata.p  	6 93
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/15-Metadata.p  	7 104
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/16-Metadata.p  	8 114
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/17-Metadata.p  	9 123
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/18-Metadata.p  	10 132
/Volumes/Biggy/Discog/artists

Current Time is Fri Apr 03, 2020 20:07:03 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/0-MediaMetadata.p 	19        179       1268      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/1-MediaMetadata.p 	30        282       2121      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/10-MediaMetadata.p 	45        474       3363      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/11-MediaMetadata.p 	59        620       4346      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/12-MediaMetadata.p 	72        809       5418      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/13-MediaMetadata.p 	83        963       6271      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/14-MediaMetadata.p 	93        1075      7041      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/15-MediaMetadata.p 	104       1192      7814      
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/16-MediaMetadata.p 	114       1291      8476      


/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/93-MediaMetadata.p 	993       11619     69119     
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/94-MediaMetadata.p 	1011      11805     70430     
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/95-MediaMetadata.p 	1024      11945     71322     
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/96-MediaMetadata.p 	1037      12099     72347     
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/97-MediaMetadata.p 	1052      12276     73548     
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/98-MediaMetadata.p 	1068      12454     74758     
/Volumes/Biggy/Discog/artists-rockcorner-db/metadata/99-MediaMetadata.p 	1078      12531     75347     



Saving 1078 entries to /Volumes/Biggy/Discog/db-rockcorner/ArtistIDToAlbumNames.p

Saving data to /Volumes/Biggy/Discog/db-rockcorner/ArtistIDToAlbumNames.p
  --> This file is 1.7MB.
Saved data to /Volumes/Biggy/Discog/db-rockcorner/ArtistIDToAlbumNames.p
  --> This fi

,Albums
114396000,"{'Albums': {'799619505': 'Hell To Pay', '57489..."
150110900,"{'Albums': {'985030875': 'Some Nights', '49644..."
153352100,"{'Albums': {'190618332': 'The CBS Recordings, ..."
214126900,{'Albums': {'959493591': 'The Green Car Motel'...
257073300,{'Albums': {'383248847': 'Dimmi cosa succede s...


***
***
***

## Artist ID --> Genre, Style, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discDC)

## Album ID --> Name, Ref, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

In [ ]:

discArtistLMAlbumsDB = getArtistAlbumsDB(discRM, force=False)

In [ ]:
from collections import Counter
cntr = Counter()
for artistData in discArtistLMAlbumsDB["Albums"]:
    for key in artistData.keys():
        cntr[key] += 1
cntr

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [ ]:
from masterdb import getArtistDB

In [ ]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

## Artist Metadata DB

In [ ]:
from masterdb import getArtistMetadataDB

In [ ]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

## Artist Albums DB

In [ ]:
from masterdb import getArtistAlbumsDB

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

## Albums DB

In [ ]:
from masterdb import getAlbumDB

In [ ]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

## Artist Album ID --> Known Albums

In [ ]:
from masterdb import getArtistAlbumKnownDB

In [ ]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random